In [1]:
!pip install findspark

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = (
    SparkSession
        .builder
        .appName("OTUS")
        .getOrCreate()
)

In [3]:
!hdfs dfs -mkdir -p /user/ubuntu/data

In [4]:
!hadoop distcp s3a://otus-mlops-source-data/2019-08-22.txt /user/ubuntu/data/2019-08-22.txt

2025-12-10 13:07:04,417 INFO tools.DistCp: Input Options: DistCpOptions{atomicCommit=false, syncFolder=false, deleteMissing=false, ignoreFailures=false, overwrite=false, append=false, useDiff=false, useRdiff=false, fromSnapshot=null, toSnapshot=null, skipCRC=false, blocking=true, numListstatusThreads=0, maxMaps=20, mapBandwidth=0.0, copyStrategy='uniformsize', preserveStatus=[], atomicWorkPath=null, logPath=null, sourceFileListing=null, sourcePaths=[s3a://otus-mlops-source-data/2019-08-22.txt], targetPath=/user/ubuntu/data/2019-08-22.txt, filtersFile='null', blocksPerChunk=0, copyBufferSize=8192, verboseLog=false, directWrite=false}, sourcePaths=[s3a://otus-mlops-source-data/2019-08-22.txt], targetPathExists=false, preserveRawXattrsfalse
2025-12-10 13:07:04,526 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2025-12-10 13:07:04,669 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2025-12-10 13:07:04,669 INFO impl.MetricsSystemImp

In [5]:
!hdfs dfs -ls data

Found 1 items
-rw-r--r--   1 ubuntu hadoop 2807409271 2025-12-10 13:07 data/2019-08-22.txt


In [6]:
#bucket = "oa-otus-bucket-b1gfn58hulkq64deu28q"
#file_path = f"s3a://{bucket}/2019-08-22.txt"
file_path = f"data/2019-08-22.txt"

# Читаем текст, пропускаем первую строку
lines = spark.sparkContext.textFile(file_path)
indexed = lines.zipWithIndex().filter(lambda x: x[1] > 0).map(lambda x: x[0])

# 🔥 Оборачиваем каждую строку в кортеж (обязательно!)
rdd_of_tuples = indexed.map(lambda line: (line,))

df_clean = spark.createDataFrame(rdd_of_tuples, ["value"])

# Парсим
df = df_clean.select(split(col("value"), ",").alias("cols")).select(
    col("cols")[0].cast("long").alias("tranaction_id"),
    col("cols")[1].cast("string").alias("tx_datetime"),
    col("cols")[2].cast("long").alias("customer_id"),
    col("cols")[3].cast("long").alias("terminal_id"),
    col("cols")[4].cast("double").alias("tx_amount"),
    col("cols")[5].cast("long").alias("tx_time_seconds"),
    col("cols")[6].cast("long").alias("tx_time_days"),
    col("cols")[7].cast("int").alias("tx_fraud"),
    col("cols")[8].cast("int").alias("tx_fraud_scenario")
)

df.cache()
print(f"Загружено {df.count():,} строк")

Загружено 46,988,418 строк


In [7]:
print("🔍 Анализ качества данных...\n")

# 1. Проверка на NULL (результат приведения типов)
null_counts = df.select([
    sum(col(c).isNull().cast("int")).alias(c) for c in df.columns
]).collect()[0]

print("❗ Пропущенные или некорректные значения (после cast):")
for col_name in df.columns:
    cnt = null_counts[col_name]
    if cnt > 0:
        print(f"  • {col_name}: {cnt:,}")

# 2. Логические проверки
checks = {
    "tranaction_id < 0": df.filter(col("tranaction_id") < 0).count(),
    "tx_amount <= 0": df.filter(col("tx_amount") <= 0).count(),
    "tx_time_seconds < 0": df.filter(col("tx_time_seconds") < 0).count(),
    "tx_time_days < 0": df.filter(col("tx_time_days") < 0).count(),
    "tx_fraud not in (0,1)": df.filter(~col("tx_fraud").isin([0, 1])).count(),
    "tx_fraud_scenario < 0": df.filter(col("tx_fraud_scenario") < 0).count(),
}

print("\n❗ Логические нарушения:")
for desc, cnt in checks.items():
    if cnt > 0:
        print(f"  • {desc}: {cnt:,}")

# 3. Некорректные даты
df_parsed = df.withColumn(
    "parsed_dt",
    to_timestamp(col("tx_datetime"), "yyyy-MM-dd HH:mm:ss")
)
invalid_dates = df_parsed.filter(col("parsed_dt").isNull()).count()
if invalid_dates > 0:
    print(f"  • Некорректные даты: {invalid_dates:,}")

# 4. Противоречие: мошенничество без сценария
bad_fraud = df.filter((col("tx_fraud") == 1) & (col("tx_fraud_scenario") == 0)).count()
if bad_fraud > 0:
    print(f"  • tx_fraud=1, но tx_fraud_scenario=0: {bad_fraud:,}")

# 5. Дубликаты по tranaction_id
duplicates = df.groupBy("tranaction_id").count().filter(col("count") > 1).count()
if duplicates > 0:
    print(f"  • Дублирующихся ID транзакций: {duplicates:,}")

# 6. Статистика
stats = df.agg(
    sum("tx_fraud").alias("fraud_count"),
    countDistinct("customer_id").alias("customers"),
    countDistinct("terminal_id").alias("terminals"),
    min("tx_amount").alias("min_amt"),
    max("tx_amount").alias("max_amt")
).collect()[0]

print("\n📊 Статистика:")
print(f"  Всего транзакций: {df.count():,}")
print(f"  Мошеннических: {int(stats['fraud_count']):,}")
print(f"  Уникальных клиентов: {stats['customers']:,}")
print(f"  Уникальных терминалов: {stats['terminals']:,}")
print(f"  Сумма: от {stats['min_amt']:.2f} до {stats['max_amt']:.2f}")

🔍 Анализ качества данных...

❗ Пропущенные или некорректные значения (после cast):

❗ Логические нарушения:
  • tx_amount <= 0: 884
  • Некорректные даты: 100
  • Дублирующихся ID транзакций: 181

📊 Статистика:
  Всего транзакций: 46,988,418
  Мошеннических: 2,527,005
  Уникальных клиентов: 988,545
  Уникальных терминалов: 1,006
  Сумма: от 0.00 до 3773.34


In [8]:
bad_rows = df.filter(
    col("tranaction_id").isNull() |
    (col("tx_amount") <= 0) |
    (~col("tx_fraud").isin([0, 1]))
)

# Сохрани в Object Storage (замените bucket)
bad_rows.write.mode("overwrite").option("header", "true").csv("s3a://oa-otus-bucket-b1gfn58hulkq64deu28q/bad_transactions")